# Chapter 2

This notebook contains the commands that are shown in the lectures.

In [1]:
import pandas as pd
import numpy as np

# Simple data operations

## Loading data from CSVs

[pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [2]:
atp_players = pd.read_csv('../data/atp_players.csv', names=['player_id', 'first_name', 'last_name', 'hand', 'birth_date', 'country_code'])
atp_players.head()

,player_id,first_name,last_name,hand,birth_date,country_code
0,100001,Gardnar,Mulloy,R,19131122.0,USA
1,100002,Pancho,Segura,R,19210620.0,ECU
2,100003,Frank,Sedgman,R,19271002.0,AUS
3,100004,Giuseppe,Merlo,R,19271011.0,ITA
4,100005,Richard Pancho,Gonzales,R,19280509.0,USA


In [3]:
print(atp_players.dtypes)

player_id         int64
first_name       object
last_name        object
hand             object
birth_date      float64
country_code     object
dtype: object


In [4]:
atp_players.head()

,player_id,first_name,last_name,hand,birth_date,country_code
0,100001,Gardnar,Mulloy,R,19131122.0,USA
1,100002,Pancho,Segura,R,19210620.0,ECU
2,100003,Frank,Sedgman,R,19271002.0,AUS
3,100004,Giuseppe,Merlo,R,19271011.0,ITA
4,100005,Richard Pancho,Gonzales,R,19280509.0,USA


## Creating and removing columns

[pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)

In [5]:
atp_players.loc[:,'birth_date'] = pd.to_datetime(atp_players.loc[:,'birth_date'], format='%Y%m%d', errors='coerce')
print(atp_players.dtypes)

player_id                int64
first_name              object
last_name               object
hand                    object
birth_date      datetime64[ns]
country_code            object
dtype: object


In [6]:
atp_players['name'] = atp_players.loc[:,'last_name'] + ', ' + atp_players.loc[:,'first_name']

atp_players.head()

,player_id,first_name,last_name,hand,birth_date,country_code,name
0,100001,Gardnar,Mulloy,R,1913-11-22,USA,"Mulloy, Gardnar"
1,100002,Pancho,Segura,R,1921-06-20,ECU,"Segura, Pancho"
2,100003,Frank,Sedgman,R,1927-10-02,AUS,"Sedgman, Frank"
3,100004,Giuseppe,Merlo,R,1927-10-11,ITA,"Merlo, Giuseppe"
4,100005,Richard Pancho,Gonzales,R,1928-05-09,USA,"Gonzales, Richard Pancho"


[drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html)

In [7]:
atp_players.drop(['first_name','last_name'], axis=1, inplace=True)
atp_players.dtypes

player_id                int64
hand                    object
birth_date      datetime64[ns]
country_code            object
name                    object
dtype: object

## Turning input processing tasks into functions

In [8]:
def load_atp_players(atp_players_file):
    atp_players = pd.read_csv(atp_players_file, names=['player_id', 'first_name', 'last_name', 'hand', 'birth_date', 'country_code'])
    atp_players.loc[:,'birth_date'] = pd.to_datetime(atp_players.loc[:,'birth_date'], format='%Y%m%d', errors='coerce')
    atp_players['name'] = atp_players.loc[:,'last_name'] + ', ' + atp_players.loc[:,'first_name']
    atp_players.drop(['first_name','last_name'], axis=1, inplace=True)
    return atp_players

atp_players = load_atp_players('../data/atp_players.csv')
atp_players.head()

,player_id,hand,birth_date,country_code,name
0,100001,R,1913-11-22,USA,"Mulloy, Gardnar"
1,100002,R,1921-06-20,ECU,"Segura, Pancho"
2,100003,R,1927-10-02,AUS,"Sedgman, Frank"
3,100004,R,1927-10-11,ITA,"Merlo, Giuseppe"
4,100005,R,1928-05-09,USA,"Gonzales, Richard Pancho"


## Categorical data format

In [9]:
atp_players_categorized = atp_players.copy()
print(atp_players_categorized['hand'].nbytes)
atp_players_categorized.loc[:,['country_code', 'hand']] = atp_players_categorized.loc[:, ['country_code', 'hand']].apply(lambda x: x.astype('category'))
print(atp_players_categorized['country_code'].nbytes)
print(atp_players_categorized['hand'].cat.categories)
atp_players_categorized.dtypes

439504
111556
Index(['A', 'L', 'R', 'U'], dtype='object')


player_id                int64
hand                  category
birth_date      datetime64[ns]
country_code          category
name                    object
dtype: object

In [10]:
def categorize_players(players):
    players.loc[:,['country_code', 'hand']] = players.loc[:, ['country_code', 'hand']].apply(lambda x: x.astype('category'))
    return players

print(atp_players.dtypes)
atp_players = categorize_players(atp_players)
atp_players.dtypes

player_id                int64
hand                    object
birth_date      datetime64[ns]
country_code            object
name                    object
dtype: object


player_id                int64
hand                  category
birth_date      datetime64[ns]
country_code          category
name                    object
dtype: object

## Joining datasets together

In [11]:
def load_atp_rankings(atp_rankings_file):
    atp_rankings = pd.read_csv(atp_rankings_file)
    atp_rankings.loc[:,'ranking_date'] = pd.to_datetime(atp_rankings.loc[:, 'ranking_date'], format='%Y%m%d', errors='coerce')
    return atp_rankings

atp_rankings00 = load_atp_rankings('../data/atp_rankings_00s.csv')
atp_rankings10 = load_atp_rankings('../data/atp_rankings_10s.csv')

print(atp_rankings00.head())
print(atp_rankings10.head())

  ranking_date  rank  player  points
0   2000-01-10     1  101736  4135.0
1   2000-01-10     2  102338  2915.0
2   2000-01-10     3  101948  2419.0
3   2000-01-10     4  103017  2184.0
4   2000-01-10     5  102856  2169.0
  ranking_date  rank  player   points
0   2010-01-04     1  103819  10550.0
1   2010-01-04     2  104745   9205.0
2   2010-01-04     3  104925   8310.0
3   2010-01-04     4  104918   7030.0
4   2010-01-04     5  105223   6785.0


[concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)

In [12]:
print(atp_rankings00.shape)
print(atp_rankings10.shape)
atp_rankings = pd.concat([atp_rankings00, atp_rankings10], ignore_index=True)
print(atp_rankings.shape)
atp_rankings.head()

(920907, 4)
(916296, 4)
(1837203, 4)


,ranking_date,rank,player,points
0,2000-01-10,1,101736,4135.0
1,2000-01-10,2,102338,2915.0
2,2000-01-10,3,101948,2419.0
3,2000-01-10,4,103017,2184.0
4,2000-01-10,5,102856,2169.0


[rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)

In [13]:
atp_rankings.rename(columns={'player':'player_id'}, inplace=True)
atp_rankings.head()

,ranking_date,rank,player_id,points
0,2000-01-10,1,101736,4135.0
1,2000-01-10,2,102338,2915.0
2,2000-01-10,3,101948,2419.0
3,2000-01-10,4,103017,2184.0
4,2000-01-10,5,102856,2169.0


In [14]:
def load_multiple_atp_rankings(atp_rankings_files):
    datasets = []
    for atp_ranking_file in atp_rankings_files:
        dataset = load_atp_rankings(atp_ranking_file)
        datasets.append(dataset)
    atp_rankings = pd.concat(datasets, ignore_index=True)
    atp_rankings.rename(columns={'player':'player_id'}, inplace=True)
    return atp_rankings

atp_rankings = load_multiple_atp_rankings(['../data/atp_rankings_00s.csv','../data/atp_rankings_10s.csv'])
print(atp_rankings.shape)
atp_rankings.head()

(1837203, 4)


,ranking_date,rank,player_id,points
0,2000-01-10,1,101736,4135.0
1,2000-01-10,2,102338,2915.0
2,2000-01-10,3,101948,2419.0
3,2000-01-10,4,103017,2184.0
4,2000-01-10,5,102856,2169.0


[merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge)

In [15]:
atp_data = atp_rankings.merge(atp_players, on='player_id', how='left')
print(atp_data.dtypes)
atp_data.head()

ranking_date    datetime64[ns]
rank                     int64
player_id                int64
points                 float64
hand                  category
birth_date      datetime64[ns]
country_code          category
name                    object
dtype: object


,ranking_date,rank,player_id,points,hand,birth_date,country_code,name
0,2000-01-10,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre"
1,2000-01-10,2,102338,2915.0,R,1974-02-18,RUS,"Kafelnikov, Yevgeny"
2,2000-01-10,3,101948,2419.0,R,1971-08-12,USA,"Sampras, Pete"
3,2000-01-10,4,103017,2184.0,R,1977-07-05,GER,"Kiefer, Nicolas"
4,2000-01-10,5,102856,2169.0,R,1976-09-10,BRA,"Kuerten, Gustavo"


## Demonstrating ATP dataset: Longest reign at rank 1

[Pandas indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#different-choices-for-indexing)

[loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html)

In [16]:
atp_top1 = atp_data.loc[atp_data.loc[:,'rank']==1].copy()
atp_top1.head()

,ranking_date,rank,player_id,points,hand,birth_date,country_code,name
0,2000-01-10,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre"
1572,2000-01-17,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre"
3143,2000-01-24,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre"
4713,2000-01-31,1,101736,5045.0,R,1970-04-29,USA,"Agassi, Andre"
6287,2000-02-07,1,101736,5045.0,R,1970-04-29,USA,"Agassi, Andre"


[shift](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html)

In [17]:
atp_top1.loc[:, 'previous_top'] = atp_top1['player_id'].shift(1)
atp_top1.head()

,ranking_date,rank,player_id,points,hand,birth_date,country_code,name,previous_top
0,2000-01-10,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre",NaN
1572,2000-01-17,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre",101736.0
3143,2000-01-24,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre",101736.0
4713,2000-01-31,1,101736,5045.0,R,1970-04-29,USA,"Agassi, Andre",101736.0
6287,2000-02-07,1,101736,5045.0,R,1970-04-29,USA,"Agassi, Andre",101736.0


In [18]:
atp_top1_reigns = atp_top1.loc[atp_top1['player_id'] != atp_top1['previous_top'],:].copy()
atp_top1_reigns.head()

,ranking_date,rank,player_id,points,hand,birth_date,country_code,name,previous_top
0,2000-01-10,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre",NaN
55359,2000-09-11,1,101948,3739.0,R,1971-08-12,USA,"Sampras, Pete",101736.0
71523,2000-11-20,1,103498,3920.0,R,1980-01-27,RUS,"Safin, Marat",101948.0
74761,2000-12-04,1,102856,4195.0,R,1976-09-10,BRA,"Kuerten, Gustavo",103498.0
87617,2001-01-29,1,103498,4265.0,R,1980-01-27,RUS,"Safin, Marat",102856.0


[diff](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.diff.html)

In [19]:
atp_top1_reigns['reign_length'] = atp_top1_reigns.loc[:,'ranking_date'].diff().shift(-1)
atp_top1_reigns.head()

,ranking_date,rank,player_id,points,hand,birth_date,country_code,name,previous_top,reign_length
0,2000-01-10,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre",NaN,245 days
55359,2000-09-11,1,101948,3739.0,R,1971-08-12,USA,"Sampras, Pete",101736.0,70 days
71523,2000-11-20,1,103498,3920.0,R,1980-01-27,RUS,"Safin, Marat",101948.0,14 days
74761,2000-12-04,1,102856,4195.0,R,1976-09-10,BRA,"Kuerten, Gustavo",103498.0,56 days
87617,2001-01-29,1,103498,4265.0,R,1980-01-27,RUS,"Safin, Marat",102856.0,28 days


[sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)

In [20]:
atp_top1_reigns.sort_values('reign_length', ascending=False).head(5)

,ranking_date,rank,player_id,points,hand,birth_date,country_code,name,previous_top,reign_length
346974,2004-02-02,1,103819,5225.0,R,1981-08-08,SUI,"Federer, Roger",104053.0,1659 days
1331200,2014-07-07,1,104925,13130.0,R,1987-05-22,SRB,"Djokovic, Novak",104745.0,854 days
155503,2001-11-19,1,103720,4365.0,R,1981-02-24,AUS,"Hewitt, Lleyton",102856.0,525 days
960449,2010-06-07,1,104745,8700.0,L,1986-06-03,ESP,"Nadal, Rafael",103819.0,392 days
1050927,2011-07-04,1,104925,13285.0,R,1987-05-22,SRB,"Djokovic, Novak",104745.0,371 days


# Using binary data formats to improve your pipeline

## CSVs and challenges with them

In [21]:
atp_data.to_csv('../data/atp_data_python.csv')
pd.read_csv('../data/atp_data_python.csv').head()

,Unnamed: 0,ranking_date,rank,player_id,points,hand,birth_date,country_code,name
0,0,2000-01-10,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre"
1,1,2000-01-10,2,102338,2915.0,R,1974-02-18,RUS,"Kafelnikov, Yevgeny"
2,2,2000-01-10,3,101948,2419.0,R,1971-08-12,USA,"Sampras, Pete"
3,3,2000-01-10,4,103017,2184.0,R,1977-07-05,GER,"Kiefer, Nicolas"
4,4,2000-01-10,5,102856,2169.0,R,1976-09-10,BRA,"Kuerten, Gustavo"


## Serialized objects

In [22]:
atp_data.to_pickle('../data/atp_data.pickle.gz')
pd.read_pickle('../data/atp_data.pickle.gz').head()

,ranking_date,rank,player_id,points,hand,birth_date,country_code,name
0,2000-01-10,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre"
1,2000-01-10,2,102338,2915.0,R,1974-02-18,RUS,"Kafelnikov, Yevgeny"
2,2000-01-10,3,101948,2419.0,R,1971-08-12,USA,"Sampras, Pete"
3,2000-01-10,4,103017,2184.0,R,1977-07-05,GER,"Kiefer, Nicolas"
4,2000-01-10,5,102856,2169.0,R,1976-09-10,BRA,"Kuerten, Gustavo"


## Feather

In [23]:
atp_data.to_feather('../data/atp_data_python.feather')
pd.read_feather('../data/atp_data_python.feather').head()

,ranking_date,rank,player_id,points,hand,birth_date,country_code,name
0,2000-01-10,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre"
1,2000-01-10,2,102338,2915.0,R,1974-02-18,RUS,"Kafelnikov, Yevgeny"
2,2000-01-10,3,101948,2419.0,R,1971-08-12,USA,"Sampras, Pete"
3,2000-01-10,4,103017,2184.0,R,1977-07-05,GER,"Kiefer, Nicolas"
4,2000-01-10,5,102856,2169.0,R,1976-09-10,BRA,"Kuerten, Gustavo"


## Parquet

In [24]:
atp_data.to_parquet('../data/atp_data_python.parquet')
pd.read_parquet('../data/atp_data_python.parquet').head()

,ranking_date,rank,player_id,points,hand,birth_date,country_code,name
0,2000-01-10,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre"
1,2000-01-10,2,102338,2915.0,R,1974-02-18,RUS,"Kafelnikov, Yevgeny"
2,2000-01-10,3,101948,2419.0,R,1971-08-12,USA,"Sampras, Pete"
3,2000-01-10,4,103017,2184.0,R,1977-07-05,GER,"Kiefer, Nicolas"
4,2000-01-10,5,102856,2169.0,R,1976-09-10,BRA,"Kuerten, Gustavo"


## HDF5

In [25]:
atp_data.to_hdf('../data/atp_data_python.h5', '/atp_data', format='table')
pd.read_hdf('../data/atp_data_python.h5','/atp_data').head()

,ranking_date,rank,player_id,points,hand,birth_date,country_code,name
0,2000-01-10,1,101736,4135.0,R,1970-04-29,USA,"Agassi, Andre"
1,2000-01-10,2,102338,2915.0,R,1974-02-18,RUS,"Kafelnikov, Yevgeny"
2,2000-01-10,3,101948,2419.0,R,1971-08-12,USA,"Sampras, Pete"
3,2000-01-10,4,103017,2184.0,R,1977-07-05,GER,"Kiefer, Nicolas"
4,2000-01-10,5,102856,2169.0,R,1976-09-10,BRA,"Kuerten, Gustavo"


## Other data formats

### Excel spreadsheets

In [26]:
efw = pd.read_excel('../data/efw.xlsx', skiprows=4, header=0, usecols='B:BU', nrows=4050)
efw.head()

,Year,ISO_Code,Countries,Economic Freedom Summary Index,Rank,Quartile,Government consumption,data,Transfers and subsidies,data.1,...,Conscription,Labor market regulations,Administrative requirements,Regulatory Burden,Starting a business,Impartial Public Administration,Licensing restrictions,Tax compliance,Business regulations,Regulation
0,2018,ALB,Albania,7.80,26.0,1.0,8.155882,12.270000,6.738420,12.470000,...,10.0,6.717929,5.651538,6.666667,9.742477,5.396,5.621940,7.175250,6.708979,7.721734
1,2018,DZA,Algeria,4.97,157.0,4.0,3.220588,29.050000,7.817129,8.511137,...,3.0,5.645397,4.215154,2.444444,9.305002,3.906,8.771111,7.029528,5.945207,5.563704
2,2018,AGO,Angola,4.75,159.0,4.0,7.698695,13.824437,9.623978,1.880000,...,0.0,5.338186,2.937894,2.444444,8.730805,5.044,7.916416,6.782923,5.642747,5.386200
3,2018,ARG,Argentina,5.78,144.0,4.0,5.938235,19.810000,6.307902,14.050000,...,10.0,5.119549,2.714233,6.666667,9.579288,7.202,5.726521,6.508295,6.399500,5.757401
4,2018,ARM,Armenia,7.92,18.0,1.0,7.717647,13.760000,7.711172,8.900000,...,0.0,6.461113,5.170406,6.000000,9.863530,6.298,9.302574,7.040738,7.279208,7.762321


### SQL databases

In [27]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///../data/atp_players_python.sqlite')

atp_players.to_sql('atp_players', engine, if_exists='replace')

pd.read_sql('atp_players', engine)

,index,player_id,hand,birth_date,country_code,name
0,0,100001,R,1913-11-22,USA,"Mulloy, Gardnar"
1,1,100002,R,1921-06-20,ECU,"Segura, Pancho"
2,2,100003,R,1927-10-02,AUS,"Sedgman, Frank"
3,3,100004,R,1927-10-11,ITA,"Merlo, Giuseppe"
4,4,100005,R,1928-05-09,USA,"Gonzales, Richard Pancho"
...,...,...,...,...,...,...
54933,54933,209899,U,NaT,RUS,"Simakin, Ilia"
54934,54934,209900,U,NaT,RUS,"Galimardanov, Oscar"
54935,54935,209901,U,NaT,RUS,"Stepin, Alexander"
54936,54936,209902,U,NaT,RUS,"Trunov, Igor"
